In [377]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
#import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import re
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation , Convolution1D
from keras.layers.embeddings import Embedding
import keras 
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Merge
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob
import re
import re
from sklearn.model_selection import train_test_split

In [ ]:
#****VIP
#**WORD TO VEC DICTIONARY
#Forming a dictionary-word2vec
folder_path= '/Users/s0c02nj/Desktop/cc.fr.300.vec'
f=open(folder_path)
doc=f.readlines()

word2vec={}
key=[]
#looping though the doc.in the doc the entire thing is saved and is separated by a space bar.
for line in doc:
    #parts contains every word separately for doc1
    parts=line.split(' ')
    #part[0] contains the word
    word=parts[0]
    key.append(word.lower())
    #embed contains the vector
    embed=np.array(parts[1:],dtype='float32')
    #filling up the dictionary
    word2vec[word]=embed


In [219]:
train=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/train.csv')
test=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/test.csv')
sample_submission=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/sample_submission.csv')

In [220]:
train_copy=train.copy()

In [221]:
#Joining Available Columns in Test and Train for Pre-processing
join=train_copy.append(test,sort=False)
join.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...


In [222]:
#Time 
join['Date-received'] =  pd.to_datetime(join['Date-received'], format='%m/%d/%Y')
join['Date-sent-to-company'] =  pd.to_datetime(join['Date-sent-to-company'], format='%m/%d/%Y')

In [223]:
#Date time diff
join['diff_time']=join['Date-sent-to-company']-join['Date-received']
join['diff_time']=join['diff_time'].apply(lambda x:x.days )

In [224]:
#Company Respose--- Mode
join['Company-response'].fillna(value=join['Company-response'].value_counts().index[0],inplace =True)
#join['Company-response'].fillna(value='No response',inplace =True)


#Missing Dispute-- Mode
join['Consumer-disputes'].fillna(value=join['Consumer-disputes'].value_counts().index[0],inplace =True)

In [225]:
#Standard Scaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [226]:
#Time Difference
diff_time=np.array(join['diff_time']).reshape(-1,1)

#Standadizing time
scaler.fit(diff_time)
enc_time= scaler.transform(diff_time)

In [359]:
#Label Encoding the Categorical Varaible
le1= preprocessing.LabelEncoder()
le1.fit(join['Transaction-Type'])
enc_trans_type= le1.transform(join['Transaction-Type'])

le2= preprocessing.LabelEncoder()
le2.fit(join['Complaint-reason'])
enc_complain_reason= le2.transform(join['Complaint-reason'])

le3= preprocessing.LabelEncoder()
le3.fit(join['Company-response'])
enc_comp_response= le3.transform(join['Company-response'])

le4= preprocessing.LabelEncoder()
le4.fit(join['Consumer-disputes'])
enc_cons_dispute= le4.transform(join['Consumer-disputes'])

le5= preprocessing.LabelEncoder()
le5.fit(train['Complaint-Status'])
y_cat= le5.transform(train['Complaint-Status'])


In [360]:
from keras.utils import to_categorical
y_enc=to_categorical(y_cat,num_classes=5)

In [327]:
#Pre-processing
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
test['Consumer-complaint-summary'] = test['Consumer-complaint-summary'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: str(x))
test['Consumer-complaint-summary'] = test['Consumer-complaint-summary'].apply(lambda x: str(x))

train['Consumer-complaint-summary']=train['Consumer-complaint-summary'].str.replace('XX','')
test['Consumer-complaint-summary']=test['Consumer-complaint-summary'].str.replace('XX','')

train['Consumer-complaint-summary']=train['Consumer-complaint-summary'].apply(lambda x:re.sub('\s+', ' ', x).strip())
test['Consumer-complaint-summary']=test['Consumer-complaint-summary'].apply(lambda x:re.sub('\s+', ' ', x).strip())

train['Consumer-complaint-summary']=train['Consumer-complaint-summary'].apply(lambda x:x.lower())
test['Consumer-complaint-summary']=test['Consumer-complaint-summary'].apply(lambda x:x.lower())

In [339]:
max_features = 80000
tokenizer = Tokenizer(lower = True, filters='', num_words=max_features)
full_text = list(join['Consumer-complaint-summary'].values)
tokenizer.fit_on_texts(full_text)

In [340]:
#Tokenizing the documents---- convert to strings
train_tokenized = tokenizer.texts_to_sequences(train['Consumer-complaint-summary'].fillna('missing'))
test_tokenized=   tokenizer.texts_to_sequences(test ['Consumer-complaint-summary'].fillna('missing'))

In [354]:
#Padding the same
max_len = 500
X_train = pad_sequences(train_tokenized, maxlen = max_len,padding='post')
X_test = pad_sequences(test_tokenized, maxlen = max_len,padding='post')

In [342]:
#word_index is dictionary of the words and the sequence
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 60526 unique tokens.


In [347]:
#Embedding matrix creation
nb_words = min(max_features, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, 300))

for word, i in word_index.items():
    #print i
    if i >= nb_words:
        continue
    if word in word2vec:
        embedding_vector = word2vec[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 1


In [348]:
#Transaction
enc_trans_type_train=enc_trans_type[0:43266]
enc_trans_type_test=enc_trans_type[43266:]

#Complaint reason
enc_complain_reason_train=enc_complain_reason[0:43266]
enc_complain_reason_test=enc_complain_reason[43266:]

enc_comp_response_train=enc_comp_response[0:43266]
enc_comp_response_test= enc_comp_response[43266:]

enc_cons_dispute_train=enc_cons_dispute[0:43266]
enc_cons_dispute_test= enc_cons_dispute[43266:]

enc_time_train=enc_time[0:43266]
enc_time_test=enc_time[43266:]

tf_idf_arr_train=tf_idf_arr[0:43266]
tf_idf_arr_test=tf_idf_arr[43266:]

In [433]:
def model_lstm():
    
    #Defining the input-----> Transaction
    inputs1 = Input(shape=(1,))
    layer_transaction = Embedding(18 , 7 ,input_length=1)(inputs1)
    layer_transaction = Flatten()(layer_transaction)
    
    #Defining the input-----> Complaint-reason
    inputs2 = Input(shape=(1,))
    layer_complain_reason = Embedding(152, 60, input_length=1)(inputs2)
    layer_complain_reason = Flatten()(layer_complain_reason)
    
    #Defining the input-----> Complaint-reason
    inputs3 = Input(shape=(1,))
    layer_company_response = Embedding(11 ,5,input_length=1)(inputs3)
    layer_company_response = Flatten()(layer_company_response)
    
    #Defining the input-----> Consumer-disputes
    inputs4 = Input(shape=(1,))
    layer_consumer_disputes = Embedding(2 ,2,input_length=1)(inputs4)
    layer_consumer_disputes = Flatten()(layer_consumer_disputes)
    
    #Defining the input ----> Time
    inputs5 = Input(shape=(1,))
    
    #Defining the input-----> Consumer-complaint-summary
    inputs6 = Input(shape=(max_len,))
    layer_complaint =  Embedding(nb_words, 300, input_length=max_len,weights = [embedding_matrix],trainable=False)(inputs6)
    
    #LSTM
    layer_lstm = LSTM(128, return_sequences=True)(layer_complaint)
    
    #Convolution layer
    conv_layer = Convolution1D(128, 2, activation='relu')(layer_lstm)
    conv_layer = MaxPooling1D(499)(conv_layer)
    conv_layer = Flatten()(conv_layer)
    
    #Merge
    layer_merge= concatenate([layer_transaction, 
                              layer_complain_reason, 
                              layer_company_response, 
                              layer_consumer_disputes,
                              inputs5, 
                              conv_layer], axis=1)
    
    #Dense Layers
    layer_dense = Dense(64, activation='relu')(layer_merge)
    layer_dense = Dropout(0.3)(layer_merge)
    
    #Output Layer
    probabilities = Dense(5,activation='softmax')(layer_dense)

    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4, inputs5, inputs6], outputs=probabilities)
    return model

    

In [434]:
model=model_lstm()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_200 (InputLayer)          (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_144 (Embedding)       (None, 500, 300)     18158100    input_200[0][0]                  
__________________________________________________________________________________________________
lstm_10 (LSTM)                  (None, 500, 128)     219648      embedding_144[0][0]              
__________________________________________________________________________________________________
input_195 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_196 

In [436]:
model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr=0.001), metrics = ["accuracy"])

In [ ]:
history = model.fit([enc_trans_type_train,
                     enc_complain_reason_train,
                     enc_comp_response_train,
                     enc_cons_dispute_train,
                     enc_time_train,
                     X_train] ,y_enc, batch_size = 1024, epochs = 1, validation_split=0.1, 
                     verbose = 1)


Train on 38939 samples, validate on 4327 samples
Epoch 1/1
33792/38939 [=========================>....] - ETA: 1:05 - loss: 0.7176 - acc: 0.7913